In [0]:
# Iniciamos sesion en SPARK
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) TRANSFORMACIONES

## *Tipos de transformaciones*
Existen dos tipos de transformaciones, dependiendo de las dependencias entre las particiones de datos:

- Transformaciones **Narrow**: consisten en dependencias estrechas en las que cada partición de entrada contribuye a una única partición de salida.

- Transformaciones **Wide**: consisten en dependencias anchas de manera que varias particiones de entrada contribuyen a muchas otras particiones de salida, es decir, cada partición de salida depende de diferentes particiones de entrada. Este proceso también se conoce como shuffle, ya que Spark baraja los datos entre las particiones del clúster.

###*Transformaciones Narrow*. 

Para los siguientes ejemplo, utilizaremos el siguiente fichero de empleados.txt, que contiene para cada empleado los siguiente cinco campos:

- el nombre
- un array de ciudades donde trabaja separadas por coma
- una estructura compuesta del sexo y la edad separadas de nuevo por comas
- un array de mapas con sus destrezas y calificación
- y finalmente un array de cargos y su consiguiente array de destrezas

In [0]:
Michael|Montreal,Toronto|Male,30|DB:80|Product:Developer Lead
Will|Montreal|Male,35|Perl:85|Product:Lead,Test:Lead
Shelley|New York|Female,27|Python:80|Test:Lead,COE:Architect
Lucy|Vancouver|Female,57|Sales:89,HR:94|Sales:Lead

#### MAP. 
La transformación map aplica la función recibida a cada elemento del RDD, de manera que vamos a poder añadir una nueva columna, modificar una existente, etc...

Por ejemplo, si la entrada es un RDD que contiene [1, 2, 3, 4, 5], al hacer rdd.map(lambda x: x*2) obtendríamos un nuevo RDD cuyos valores contiene su doble, es decir, [2, 4, 6, 8, 10]:

In [0]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
resultRDD = rdd.map(lambda x: x*2)
resultRDD.collect()          # [2, 4, 6, 8, 10]

Out[20]: [2, 4, 6, 8, 10]

Para cargar un fichero desde nuestro HDD local debemos subirlo al workspace de DATABRICK a modo de tabla.
Para ello, desde la opción de menu "Catálogo" incorporaremos una "Nueva Tabla". Desde esa pantalla se podrá 
subir el fichero de texto, csv o similar y nos quedará disponible en la ruta "/FileStore/tables/<nombre_fichero.ext>"

Seguidamente podemos leerlo y transformarlo en un DataFrame (o RDD) de la siguiente forma:

In [0]:
rddLocal = spark.read.option("delimiter", "|").csv("/FileStore/tables/empleados.txt")
rddLocal.show()

+-------+----------------+---------+--------------+--------------------+
|    _c0|             _c1|      _c2|           _c3|                 _c4|
+-------+----------------+---------+--------------+--------------------+
|Michael|Montreal,Toronto|  Male,30|         DB:80|Product:Developer...|
|   Will|        Montreal|  Male,35|       Perl:85|Product:Lead,Test...|
|Shelley|        New York|Female,27|     Python:80|Test:Lead,COE:Arc...|
|   Lucy|       Vancouver|Female,57|Sales:89,HR:94|          Sales:Lead|
+-------+----------------+---------+--------------+--------------------+



In [0]:
rddLocal = sc.textFile("/FileStore/tables/empleados.txt")
print(rddLocal.count())            # 4 - cantidad de líneas
resultRDD = rddLocal.map(len)    # obtenemos la cantidad de caracteres de cada línea
resultRDD.collect()         # [61, 52, 60, 50]

4
Out[23]: [61, 52, 60, 50]

In [0]:
# Split para separación de campos aplicando 'map' usando una función lambda
resultMap = rddLocal.map(lambda x: x.split("|"))
resultMap.collect()

Out[24]: [['Michael', 'Montreal,Toronto', 'Male,30', 'DB:80', 'Product:Developer Lead'],
 ['Will', 'Montreal', 'Male,35', 'Perl:85', 'Product:Lead,Test:Lead'],
 ['Shelley', 'New York', 'Female,27', 'Python:80', 'Test:Lead,COE:Architect'],
 ['Lucy', 'Vancouver', 'Female,57', 'Sales:89,HR:94', 'Sales:Lead']]

Se ha obtenido una lista de listas por cada fila del fichero, cada lista conteniendo los cincos campos de cada empleado

#### FLATMAP. 
La transformación flatMap es muy similar a la anterior, pero en vez de devolver una lista con un elemento por cada entrada, devuelve una única lista deshaciendo las colecciones en elementos individuales:

In [0]:
rdd = sc.textFile("/FileStore/tables/empleados.txt") 
resultFM = rdd.flatMap(lambda x: x.split("|"))
resultFM.collect()

Out[26]: ['Michael',
 'Montreal,Toronto',
 'Male,30',
 'DB:80',
 'Product:Developer Lead',
 'Will',
 'Montreal',
 'Male,35',
 'Perl:85',
 'Product:Lead,Test:Lead',
 'Shelley',
 'New York',
 'Female,27',
 'Python:80',
 'Test:Lead,COE:Architect',
 'Lucy',
 'Vancouver',
 'Female,57',
 'Sales:89,HR:94',
 'Sales:Lead']

Se ha obtenido cada atributo por separado y todos dentro de la misma lista.

## FILTER

Permite el filtrado de elementos que cumplen una condición dada

In [0]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
resultRDD = rdd.filter(lambda x: x%2==0) # multiplos de dos existentes en el conjunto de elementos procesado
resultRDD.collect()

Out[28]: [2, 4]

También podemos anidar diferentes transformaciones. Para este ejemplo, vamos a crear tuplas formadas por un número y su cuadrado, y luego quitar los que no coincide el número con su cuadrado (sólo coinciden el 0 y el 1), y luego aplanarlo en una lista:

In [0]:
rdd10 = sc.parallelize(range(10+1))
rddPares = rdd10.map(lambda x: (x, x**2)).filter(lambda x: (x[0] != x[1])).flatMap(lambda x: x)
rddPares.collect()      # [2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81, 10, 100]

Out[29]: [2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81, 10, 100]

Veamos otro ejemplo. Retomamos los datos de los empleados y si queremos filtrar los empleados que son hombres, primero separamos por las | y nos quedamos con el tercer elemento que contiene el sexo y la edad. A continuación, separamos por la coma para quedarnos en el sexo en la posición 0 y la edad en el 1, y comparamos con el valor deseado:

In [0]:
rdd = sc.textFile("/FileStore/tables/empleados.txt") 
resultFM = rdd.filter(lambda x: x.split("|")[2].split(",")[0] == "Male")
resultFM.collect()

Out[32]: ['Michael|Montreal,Toronto|Male,30|DB:80|Product:Developer Lead',
 'Will|Montreal|Male,35|Perl:85|Product:Lead,Test:Lead']

In [0]:
# Podemos usar DISTINCT para eliminar repeticiones
rdd = sc.parallelize([1,1,2,2,3,4,5])
resultRDD = rdd.distinct()
resultRDD.collect()     # [4, 1, 5, 2, 3]

Out[33]: [1, 2, 3, 4, 5]

### Trabajando con conjuntos 
*UNIÓN*

In [0]:
# Uniendo dos RDD
rdd1 = sc.parallelize([1,2,3,4])
rdd2 = sc.parallelize([5,6,7,8])
resultRDD = rdd1.union(rdd2)
resultRDD.collect()     # [1, 2, 3, 4, 5, 6, 7, 8]

Out[34]: [1, 2, 3, 4, 5, 6, 7, 8]

*INTERSECCIÓN*

In [0]:
rdd1 = sc.parallelize([1,2,3,4])
rdd2 = sc.parallelize([3,4,5,6])
resultRDD = rdd1.intersection(rdd2)
resultRDD.collect()     # [3, 4]

Out[35]: [3, 4]

*SUBTRACT - RESTA*

In [0]:
rdd1 = sc.parallelize([1,2,3,4])
rdd2 = sc.parallelize([3,4,5,6])
resultRDD = rdd1.subtract(rdd2)
resultRDD.collect()     # [1, 2]

Out[36]: [1, 2]

###*EJERCICIO*###
Si tenemos dos RDD (A y B):
``` python
rddA = sc.parallelize([1,2,3,4])
rddB = sc.parallelize([3,4,5,6])
```
¿Cómo conseguimos los elementos que están en A y no B y los de B que no están en A? (es decir [1, 2, 5, 6])):

*RDD de Pares*.   
Una técnica muy común a la hora de trabajar con RDD es hacerlo con elementos que tienen el formato (clave, valor), pudiendo las claves y los valores ser de cualquier tipo.

In [0]:
listaTuplas = [(1,'a'), (2,'b'), (3,'c'), (4,'d')]
rddTuplas= sc.parallelize(listaTuplas)

Para generar un RDD de pares, además de crearlo nosotros a partir de una lista, podemos emplear las siguientes operaciones:
- zip, que une dos RDDs del mismo tamaño

In [0]:
lista1 = ['a','b','c','e','f','g','h']
lista2 = [4, 5, 6, 7, 8, 9, 10]
rddZip = sc.parallelize(lista1).zip(sc.parallelize(lista2)).collect()
# [('a', 4), ('b', 5), ('c', 6), ('e', 7), ('f', 8), ('g', 9), ('h', 10)]
print(rddZip)

rddZipSecuencia = sc.parallelize(zip(lista1,range(len(lista1)))) # usando el tamaño de la lista
# [('a', 0), ('b', 1), ('c', 2), ('e', 3), ('f', 4), ('g', 5), ('h', 6)]
rddZipSecuencia.collect()

[('a', 4), ('b', 5), ('c', 6), ('e', 7), ('f', 8), ('g', 9), ('h', 10)]
Out[43]: [('a', 0), ('b', 1), ('c', 2), ('e', 3), ('f', 4), ('g', 5), ('h', 6)]

- 
map: asignando a cada elemento un valor o cálculo sobre él mismo:

In [0]:
lista  = ['Hola', 'Adiós', 'Hasta luego']
rddMap = sc.parallelize(lista).map(lambda x: (x, len(x)))
# [('Hola', 4), ('Adiós', 5), ('Hasta luego', 11)]
rddMap.collect()

Out[46]: [('Hola', 4), ('Adiós', 5), ('Hasta luego', 11)]

- keyBy: permite crear las claves a partir de cada elemento:

In [0]:
rddKeyBy = sc.parallelize(lista).keyBy(lambda x: x[0])  # creamos una clave con la primera letra
# [('H', 'Hola'), ('A', 'Adiós'), ('H', 'Hasta luego')]
rddKeyBy.collect()

Out[48]: [('H', 'Hola'), ('A', 'Adiós'), ('H', 'Hasta luego')]

### *EJERCICIO*
A partir de la lista "Perro Gato Loro Pez León Tortuga Gallina":

1. Crea un RDD a partir de esta lista
2. Convierte el RDD normal en un RDD de pares cuya clave sea la primera letra del animal
3. Crea otro RDD de pares pero poniendo como clave un número incremental
4. ¿Y si queremos que el índice incremental empiece en 100?

Sobre los RDD de pares, podemos realizar las siguientes transformaciones:

- keys: devuelve las claves
- values: devuelve los valores
- mapValues: Aplica la función sobre los valores
- flatMapValues Aplica la función sobre los valores y los aplana.

A continuación se muestra un fragmento de código para poner en práctica las transformaciones comentadas:

In [0]:
listaTuplas = [('a',1), ('z',3), ('b',4), ('c',3), ('a',4)]
rddTuplas = sc.parallelize(listaTuplas)

claves = rddTuplas.keys()       # ['a', 'z', 'b', 'c', 'a']
valores = rddTuplas.values()    # [1, 3, 4, 3, 4]

rddMapValues = rddTuplas.mapValues(lambda x: (x,x*2))
# [('a', (1, 2)), ('z', (3, 6)), ('b', (4, 8)), ('c', (3, 6)), ('a', (4, 8))]
rddMapValues.collect()

Out[53]: [('a', (1, 2)), ('z', (3, 6)), ('b', (4, 8)), ('c', (3, 6)), ('a', (4, 8))]

In [0]:
rddFMV = rddTuplas.flatMapValues(lambda x: (x,x*2))
# [('a', 1),
#  ('a', 2),
#  ('z', 3),
#  ('z', 6),
#  ('b', 4),
# ...

rddFMV.collect()

Out[54]: [('a', 1),
 ('a', 2),
 ('z', 3),
 ('z', 6),
 ('b', 4),
 ('b', 8),
 ('c', 3),
 ('c', 6),
 ('a', 4),
 ('a', 8)]